# Wie verspätet ist der neue 8er?
Daten von Open Data Stadt Zürich
2017: https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2017
2018: https://data.stadt-zuerich.ch/dataset/vbz-fahrzeiten-ogd

In [ ]:
import pandas as pd
import requests
import time 
import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from cycler import cycler
plt.rcParams.update(
    {'pdf.fonttype':42,
    'ps.fonttype': 42
})
import numpy
import re
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import glob
from bs4 import BeautifulSoup

In [ ]:
#import all filenames of the downloaded .csv from Open Data Zurich
filenames = glob.glob('/Volumes/USB_STICK/VBZ/*.csv')
filenames

In [ ]:
df = pd.read_csv(filenames[0])
df = df[df['linie']==8]

for filename in filenames[1:]:
    dataframes = pd.read_csv(filename)
    dataframes = dataframes[dataframes['linie']==8]
    #dataframe['filename'] = filename
    df = pd.concat([df, dataframes])
    print(filename)

In [ ]:
df_acht = df

In [ ]:
#soll-ist vergleich berechnen
df_acht['differenz'] = df_acht.ist_ab_von-df_acht.soll_ab_von

In [ ]:
#importiere haltestellenliste
haltestellen = pd.read_csv('/Volumes/USB_STICK/VBZ/halte/haltestelle_2018.csv')
haltestellen.head()

In [ ]:
haltepunkte = pd.read_csv('/Volumes/USB_STICK/VBZ/halte/haltepunkt_2018.csv')

In [ ]:
haltepunkte.head()

In [ ]:
#merge haltestellen und haltepunkte
df_haltestelle = df_acht.merge(haltepunkte, left_on='halt_punkt_id_von', right_on='halt_punkt_id')
print(len(df_haltestelle))

In [ ]:
df_haltestelle = df_haltestelle.merge(haltestellen, left_on='halt_id', right_on='halt_id')
print(len(df_haltestelle))

In [ ]:
df_haltestelle.head()

#### Auswertunge der Verspätung

In [ ]:
#wenn ist-wert 120 sekunden über soll wert ist = verspätet
df_haltestelle['late'] = df_haltestelle.differenz > 119

In [ ]:
df_haltestelle['late']=df_haltestelle['late'].replace(True, 1)

In [ ]:
#datumsformat
df_haltestelle['date_created']=pd.to_datetime(df_haltestelle['betriebsdatum'], format="%d.%m.%y")

In [ ]:
#neue datensätze für jahre
df_haltestelle_2017 = df_haltestelle[(df_haltestelle.date_created>= "2017-01-01")&(df_haltestelle.date_created < "2017-12-10")]
df_haltestelle_2018 = df_haltestelle[(df_haltestelle.date_created>= "2017-12-10")&(df_haltestelle.date_created <= "2018-10-06")]

In [ ]:
#datensätze für die jeweilige richtung
df_haltestelle_2017_1 = df_haltestelle_2017[df_haltestelle_2017.richtung==1]
df_haltestelle_2017_2 = df_haltestelle_2017[df_haltestelle_2017.richtung==2]
df_haltestelle_2018_1 = df_haltestelle_2018[df_haltestelle_2018.richtung==1]
df_haltestelle_2018_2 = df_haltestelle_2018[df_haltestelle_2018.richtung==2]


In [ ]:
#leeren dataframe erstellen
haltelist = pd.DataFrame(columns=['haltestellen','j17_1','j17_2','j18_1','j18_2'])
haltelist.head()

In [ ]:
#liste der haltestellen erstellen für loop
haltelist['haltestellen'] = list(set(df_haltestelle['halt_lang']))
haltelist


In [ ]:
#funktion, welche die häufigkeit der verspätung pro haltestelle berechnet
def get_percentage(halts, data):
    try: 
        percent = data[data.halt_lang==halts].late.value_counts('normalize').values[1]*100
        return percent
    except:
        return 'error'

###### Loop über alle DF

In [ ]:
percent_list = []
for stopp in haltelist['haltestellen']:
        verspaetung = get_percentage(stopp,df_haltestelle_2017_1)
        percent_list.append(verspaetung)
haltelist['j17_1']=percent_list

In [ ]:
percent_list = []
for stopp in haltelist['haltestellen']:
        verspaetung = get_percentage(stopp,df_haltestelle_2017_2)
        percent_list.append(verspaetung)
haltelist['j17_2']=percent_list

In [ ]:
percent_list = []
for stopp in haltelist['haltestellen']:
        verspaetung = get_percentage(stopp,df_haltestelle_2018_1)
        percent_list.append(verspaetung)
haltelist['j18_1']=percent_list

In [ ]:
percent_list = []
for stopp in haltelist['haltestellen']:
        verspaetung = get_percentage(stopp,df_haltestelle_2018_2)
        percent_list.append(verspaetung)
haltelist['j18_2']=percent_list


In [ ]:
haltelist

#### Hole aktuellen Haltestellenliste
2018 Haltestellen, um Extra-, Depot- oder Umwegfahrten herauszubekommen.

In [ ]:
raw_html = requests.get("https://www.fahrplaninfo.zvv.ch/frame_linie3.php?lang=de&sel_linie=%7C01008%7C8&sel_gk=1")
soup_doc = BeautifulSoup(raw_html.content, "html.parser")
#print(soup_doc)

In [ ]:
stations = soup_doc.find_all('td')

In [ ]:
haltlist_8 = [station.text for station in stations if len(station.text) > 7]
print(len(haltlist_8))
haltlist_8

In [ ]:
haltlist_8 = haltlist_8[0:25]


In [ ]:
#ergängzen mit zwei Haltestellen, die es 2017 noch gab
haltlist_8.insert(-5, 'Zürich, Kunsthaus')
haltlist_8.insert(-5, 'Zürich, Hottingerplatz')

haltlist_8

In [ ]:
haltlist_8 = pd.DataFrame(haltlist_8)
haltlist_8.columns = ["haltestelle"]

In [ ]:
#nun mergen
verspaetungen = haltlist_8.merge(haltelist, left_on='haltestelle', right_on='haltestellen')
verspaetungen

In [ ]:
verspaetungen[['j17_1','j17_2','j18_1','j18_2']]=verspaetungen[['j17_1','j17_2','j18_1','j18_2']].replace('error', 'NA')
verspaetungen

In [ ]:
#save to csv
#verspaetungen.to_csv('achter_verspraetungen.csv', index=False)